# Lab 5 - Apache Spark - wprowadzenie.

Apache Spark jest silnikiem do przetwarzania dancych na dużą skalę, pozwalający na wykonywanie operacji w sposób zrównoleglony i rozproszony. Spark dostarcza API dla języków java, Scala, Python oraz R do przetwarzania grafów obliczeń. Spark składa się z wielu narzędzi takich jak:
* Resilient Distributed Datasets (RDD) - niskopoziomowy typ zbioru danych Spark, na którym opierają się struktury danych na wyższych poziomach abstrakcji,
* Spark SQL - Spark Dataset, Spark DataFrame,
* Pandas API on Spark - API pozwalające na wykorzystanie biblioteki pandas w sposób zrównoleglony na klastrze Spark,
* Structured Streaming - zestaw narzędzi do przetwarzania strumieniowego,
* MLlib - moduł wspierający wykorzystanie Machine Learning z użyciem typów danych Spark oraz klastrów Spark,
* GraphX - przetwarzanie grafów,
* SparkR - API w języku R do pracy w środowisku Spark,
* PySpark - API Python do pracy w środowisku Spark,
* Spark SQL CLI - przetwarzanie danych z użyciem Spark SQL z poziomu wiersza poleceń.

## 1. Uruchomienie środowiska Spark z wykorzystaniem PySpark.

W trakcie zajęc z racji dotychczasowego środowiska pracy (Python) będzie wykorzystywane API Pythona, które dostarcza Spark.

**Dokumentacja Spark Python API:** https://spark.apache.org/docs/3.5.5/api/python/index.html

> Oficjalna dokumentacja alternatywnych sposobów instalacji oraz zależności modułu PySpark: https://spark.apache.org/docs/latest/api/python/getting_started/install.html


**UWAGA!**

Uruchamiamy poniższą komórkę tylko dla konfiguracji z Dockerem!

In [ ]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

**UWAGA!**

Uruchamiamy poniższą komórkę tylko dla konfiguracji z lokalną instalacją PySpark (scenariusz 1)!

In [5]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_HOME'] = sys.executable

> Przed uruchomieniem poniższej komórki należy zainstalować moduł `pyspark` do środowiska wirtualnego Pythona

> Oficjana dokumentacja konfiguracji Sparka, również parametrów wywołania: https://spark.apache.org/docs/3.5.5/configuration.html

Aby możliwe było wysłanie zadań do wykonania z wykorzystaniem klastra Spark (lokalnego lub rozproszonego) musimy uzyskać najpierw **referencję do obiektu typu SparkSession**. Tworząc obiekt musimy zdefiniować nazwę aplikacji, którą otrzyma ta sesja oraz możemy zdefiniować dodatkowe parametry. Przykład poniżej.

In [6]:
# import findspark
from pyspark.sql import SparkSession

# findspark.init()

# local[2] - oznacza dwa executory (workery), można to zmienić w razie potrzeb dla nowej sesji
# "spark.executor.memory", "2g" - 2GB RAM dla każdego executora
# "spark.driver.memory", "1g" - 1GB RAM dla drivera
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.executor.memory", "2g") \
        .config("spark.driver.memory", "1g") \
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()

In [7]:
spark.sparkContext

<SparkContext master=local[2] appName=Create-DataFrame>

Podobnie jak w przypadku biblioteki Dask możemy śledzić pracę klastra poprzez przeglądarkę. Adres to http://\<host\>:4040. Wykorzystując dockera wymagane jest stworzenie odpowiedniego mapowania portu oraz w naszym przypadków również zmiany domyślnego hosta, który zapewne będzie skróconą wersją hasha kontenera. Skoro port jest zmapowany to znaczy, że będzie dostępny na hoście lokalnym. Finalnie więc adres dla węzła master powinien być dostępny pod adresem `http://localhost:4040`.

In [4]:
# jeżeli chcemy zatrzymać sesję Spark (lub na potrzeby jej resetu)
spark.sparkContext.stop()

## 2. Spark Resilient Distributed Datasets (RDD).

Na wysokim poziomie każda aplikacja Spark składa się z **programu sterownika (ang. driver program)**, który uruchamia główną funkcję użytkownika i wykonuje różne operacje równoległe na klastrze. 

Główną abstrakcją zapewnianą przez Spark jest **odporny rozproszony zbiór danych (Resilient Distributed Datasets - RDD)**, który jest zbiorem elementów podzielonych na węzły klastra, które mogą być obsługiwane równolegle. RDD mogą być również utrwalane w pamięci klastra, umożliwiając jego ponowne wykorzystanie w operacjach równoległych. Wreszcie, RDD automatycznie odzyskują sprawność po awarii węzła.

Wykorzystanie API RDD nie jest rekomendowane w wielu przypadkach, gdyż wymaga dobrej znajomości niskopoziomowego API Sparka i ręcznej optymalizacji operacji. Zazwyczaj lepszym pomysłem będzie wykorzystanie Spark Dataset oraz Spark DataFrame.

Kilka przykładów wykorzystania tego API zostanie jednak tutaj przedstawionych.

> Poradnik programisty dla RDD znajduje się pod linkiem: https://spark.apache.org/docs/3.5.5/rdd-programming-guide.html  
> API dla PySpark RDD: https://spark.apache.org/docs/3.5.5/api/python/reference/api/pyspark.RDD.html

In [96]:
# lista wartości zostaje podzielona na partycje i rozproszona na wszystkie dostępne węzły
# https://spark.apache.org/docs/3.5.5/api/python/reference/api/pyspark.SparkContext.parallelize.html
rdd = spark.sparkContext.parallelize(list(range(20)))

# rdd w formie rozproszonej zostaje scalone w listę zawierającą wszystkie elementy RDD
# np. za pomocą funkcji collect()
# https://spark.apache.org/docs/3.5.5/api/python/reference/api/pyspark.RDD.collect.html

rddCollect = rdd.collect()
display(type(rddCollect))
print(f"Liczba partycji: {rdd.getNumPartitions()}")
print(rddCollect)
# print(f"Pierwszy element: {rdd.first()}")
# print(f"Ile elementów: {rdd.count()}")
rdd.first()

list

Liczba partycji: 2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


0

In [86]:
# obiekt RDD może przechowywać dane z różnych źródeł, które są zgodne z systemem plików Apache Hadoop
# np. Amazon S3, Cassandra, HDFS, HBase i inne

# możemy dla uniknięcia potrzeby każdorazowego odwoływania się do kontekstu poprzez spark.sparkContext zapisać sobie to w zmiennej pomocniczej
sc = spark.sparkContext
# tutaj wczytamy do RDD plik tekstowy
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

Więcej informacji odnośnie obsługi plików w środowisku Spark można znaleźć m.in. tu: https://spark.apache.org/docs/3.5.5/rdd-programming-guide.html#external-datasets

In [10]:
display(pan_tadeusz_file.getNumPartitions())

# jaka jest struktura tego zbioru danych?
# pojedyncza wartość to linia z pliku
pan_tadeusz_file.take(10)

2

['Adam Mickiewicz',
 '',
 'Pan Tadeusz',
 'czyli ostatni zajazd na Litwie',
 '',
 'ISBN 978-83-288-2495-9',
 '',
 '',
 '',
 '']

In [87]:
# możemy zmienić liczbę automatycznie stworzonych partycji i ponownie rozproszyć je po węzłach
pan_tadeusz_file = pan_tadeusz_file.repartition(4)
pan_tadeusz_file.getNumPartitions()

# również metoda coalesce może posłużyć nam do zmiany ilości partycji dla obiektu RDD np. po zastosowaniu filtrowania, które
# znacznie zmniejsza wielkość pierwotnego obiektu RDD a co za tym idzie każdej partycji i dalsze obliczenia mogą nie być
# wykonywane zbyt efektywnie (zbyt mały rozmiar partycji)
# https://spark.apache.org/docs/3.5.5/api/python/reference/api/pyspark.RDD.coalesce.html
# główna różnica między repartition a coalesce jest taka, że ta pierwsza wykorzystuje mechanizm tasowania danych a ta druga może, ale nie
# musi go wykorzystywać gdyż możemy tym sterować za pomocą parametru wywołania tej metody

4

In [13]:
# jedną z funkcji dostępnej w tym API jest możliwość wykonania funkcji na każdej z partycji
# minusem może być to, że funkcja foreachPartition zwraca typ None, więc wyniki należy przetworzyć w inny sposób
# docs: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.foreachPartition.html

def count_words(iterator):
    words = sum([len(x.split()) for x in iterator])
    print(words)

pan_tadeusz_file.foreachPartition(count_words)

In [25]:
# w przypadku pracy w środowisku rozproszonym (cluster mode) nie zobaczymy rezultatów, gdyż zostały one wykonane na
# executorach (tutaj funkcja print)
# jeżeli chcemy jednak zobaczyć efekt, to należy najpierw pobrać dane na driver poprzez np. collect(), a następnie
# wywołać funkcję, ale to nie zadziała dla PySparka, gdyż w tym API collect() zwraca obiekt typu list, na którym
# nie możemy wywołać foreach() - to zadziałe jednak w natywnym środowisku Sparka, czyli w języku scala

Przy wypisywaniu wartości z RDD trzeba również zwrócić uwagę na różnicę w działaniu tych metod w trybie pracy lokalnej (czyli tak jak w tym labie) oraz klastra. Efekty mogą być różne, więcej przeczytasz tu: https://spark.apache.org/docs/3.5.5/rdd-programming-guide.html#printing-elements-of-an-rdd

**RDD obsługują dwa rodzaje operacji: transformacje**, które tworzą nowy zbiór danych z istniejącego, oraz **akcje**, które zwracają wartość do programu sterownika po uruchomieniu obliczeń na zbiorze danych. Przykładowo, map jest transformacją, która przepuszcza każdy element zbioru danych przez funkcję i zwraca nowy RDD reprezentujący wyniki. Z drugiej strony, reduce jest akcją, która agreguje wszystkie elementy RDD przy użyciu pewnej funkcji i zwraca końcowy wynik do programu sterownika.

Wszystkie transformacje są wykonywane w sposób **leniwy** tzn, że obliczenia nie są wykonywane dopóki nie jest potrzebnych wynik. To podobnie jak w przypadku frameworka Dask umożliwia optymalizację obliczeń np. w przypadku gdy nie są potrzebne wyniki pośrednie z każdego węzła po zastosowaniu funkcji poprzez `map`, ale tylo wynik akcji `reduce`, więc nie ma potrzeby przesyłania całych pośrednich RDD do drivera.

**Lista wybranych transformacji dostępna jest tu:** https://spark.apache.org/docs/3.5.5/rdd-programming-guide.html#transformations

**Lista wybranych akcji tu:** https://spark.apache.org/docs/3.5.5/rdd-programming-guide.html#actions

W uzasadnionych przypadkach można również przyspieszyć obliczenia poprzez utrwalenie danych w pamięci lub pamięci podręcznej poprzez metody `persist` lub `cache` na obiekcie RDD.

**Kilka przykładów transformacji i akcji**

In [33]:
# funkcje map oraz reduce

# możemy również wykonać operację w inny sposób, tym raze mapując funkcję na każdy element obiektu RDD
# zwrócony zostanie obiekt RDD, na którym możemy wykonać kolejne operacje

# transformacje są leniwe - to obiekt RDD, a nie obiekt z przeliczoną ilością wyrazów
display(pan_tadeusz_file.map(lambda s: len(s.split())))

# dzielimy każdą wartość na wyrazy (tak domyślnie zadziała split() na linii tekstu,
# a następnie wywołujemy funkcję len(), na tym co zostanie zwrócone (ilość wyrazów)
# wyświetlamy tylko pierwsze 10 wartości poprzez take(10) - i dopiero ta akcja faktycznie wyzwala obliczenia
display(pan_tadeusz_file.map(lambda s: len(s.split())).take(10))

# np. reduce - i tu nawiązanie do znanej techniki przetwarzania rozproszonego - MapReduce
# więcej: https://en.wikipedia.org/wiki/MapReduce
# oraz: https://wiadrodanych.pl/big-data/jak-dziala-mapreduce/

# początkowa akcja jest taka sama (podział i liczba wyrazów), ale w następnej kolejności
# wywołujemy reduce, które zsumuje liczbę wyrazów, tutaj funkcją anonimową (lambda)
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(lambda a, b: a + b)

PythonRDD[25] at RDD at PythonRDD.scala:53

[8, 6, 7, 5, 8, 5, 6, 4, 7, 5]

69095

In [32]:
# lub tak - ten sam efekt

# każdy operator w Pythonie ma swoją dedykowaną metodę - tu wszystkie jest obiektem
from operator import add

# wywołujemy reduce z użyciem add, które ja spojrzeć w jej sygnaturę przyjmuje dwa argumenty, i zwraca sumę (o ile to możliwe)
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(add)

69095

In [31]:
# dokumentacja dla operatora add
print(add.__doc__)

Same as a + b.


In [29]:
# różnica między map() a flatMap() dla tego przypadku
display(pan_tadeusz_file.map(lambda s: s.split()).take(10))
pan_tadeusz_file.flatMap(lambda s: s.split()).take(10)

[['I', 'zaraz', 'mogłem', 'pieszo,', 'do', 'Twych', 'świątyń', 'progu'],
 ['Iść', 'za', 'wrócone', 'życie', 'podziękować', 'Bogu),'],
 ['Tak', 'nas', 'powrócisz', 'cudem', 'na', 'Ojczyzny', 'łono.'],
 ['Tymczasem', 'przenoś', 'moją', 'duszę', 'utęsknioną'],
 ['Do', 'tych', 'pagórków', 'leśnych,', 'do', 'tych', 'łąk', 'zielonych,'],
 ['Szeroko', 'nad', 'błękitnym', 'Niemnem', 'rozciągnionych;'],
 ['Do', 'tych', 'pól', 'malowanych', 'zbożem', 'rozmaitem,'],
 ['Wyzłacanych', 'pszenicą,', 'posrebrzanych', 'żytem;'],
 ['Gdzie', 'bursztynowy', 'świerzop,', 'gryka', 'jak', 'śnieg', 'biała,'],
 ['Gdzie', 'panieńskim', 'rumieńcem', 'dzięcielina', 'pała,']]

['I',
 'zaraz',
 'mogłem',
 'pieszo,',
 'do',
 'Twych',
 'świątyń',
 'progu',
 'Iść',
 'za']

### Zadania

**Zadanie 1**  
Wykorzystując ten sam plik z treścią Pana Tadeusza policz i wyświetl:
* 1.1 - liczbę linii w tym pliku,
* 1.2 - 10 najdłuższych linii - ich długość oraz faktyczne linie - oddzielnie polecenia,
* 1.3 - listę wszystkich unikalnych wyrazów w tym pliku,
* 1.4 - kolekcję, która zawiera unikalne wyrazy i liczbę ich wystąpień w pliku,
* 1.5 - z kolekcji z punktu 1.5, wyświetl 10 najczęściej występujących wyrazów,
* 1.6 - tylko te linie, które zawierają więcej niż 7 wyrazów,
* 1.7 - wszystkie linie, które zawierają słowo 'Tadeusz',
* 1.8 - 10 ostatnich linii, ale zapisanych małymi literami,
* 1.9 - sumę znaków w tym pliku.

**Zadanie 2**  
Wykorzystując listę stopwords z adresu https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt wykorzystaj akcje i transformacje RDD i wygeneruj listę unikalnych słów z pliku z treścią Pana Tadeusza pomijając powyższe słowa stop oraz wszelkie znaki przestankowe. Wynik zapisz do słownika, a następnie do pliku json o nazwie pan_tadeusz_bag_of_words.json.
Które słowo występuje w tym tekście najczęściej? Wyświetl je z wyników wygenerowanych powyżej.

**Podpowiedzi**

W przypadku **zadania 1** większość funkcji, których trzeba użyć znajduje się w API PySpark RDD: https://spark.apache.org/docs/3.5.5/api/python/reference/api/pyspark.RDD.html

Dla konkretnych zadań (niektóre pomijam, gdyż są proste) moje sugestie (zapewne można użyć w niektórych przypadkach innych funkcji):
* 1.2 - `top()`, `take()`, `map()`, `sortBy()`
* 1.3 - `distinct()`
* 1.4 - `countByValue()`
* 1.5 - wykorzystałem sortowanie słownika z pkt. 1.4 funkcją `sorted()`, której można przekazać klucz, według, którego sortowanie powinno zostać wykonane. Tutaj chodzi o to, aby sortować po wartości z par (klucz, wartość), które znajdują się w słowniku
* 1.6, 1.7 - `filter()`
* 1.8 - tu nie ma wbudowanej funkcji, która zwraca `n` ostatnich elementów. Można zwrócić wszystkie, a następnie zamienić na małe litery i zwrócić tylko wycinek z ostatnimi 10 elementami.
* 1.9 - `map()`, `reduce()`

**zadanie 2**

Zadanie można zrealizować wykorzystując funkcje przedstawione w podpowiedziach do zadania 1. Dodatkowo:
* można napisać własną funkcję do usuwania znaków przestankowych i poprzez `map` oraz `lambdę` ją wywołać na każdym słowie,
* zapisanie słownika do pliku json jest dość proste, wykorzystaj moduł `json` oraz metodę `dumps()`,
* co do najczęściej występującego słowa - w części 1 zadań było już podobne zadanie.